# Lab 10 - Bagging Decision Trees




In [66]:
import sys
sys.path.append("../")
from utils import *
from sklearn import tree


In [40]:
train_n = 5000
test_n = 500
bootstrap_samples = 5000

samples, tags =  create_data_bagging_utils(d = 8, n_samples = train_n + test_n)
r = np.random.choice(np.arange(train_n + test_n), size= train_n + test_n, replace=False).tolist()
train_test = np.zeros(train_n + test_n)
train_test[r[train_n:]] = 1
samples_train, samples_test, tags_train, tags_test = samples[r[:train_n],:], samples[r[train_n:],:],tags[r[:train_n]], tags[r[train_n:]] 

px.scatter(x = samples_train[:,0], y =samples_train[:,1], color = tags_train).show()

Let's create bootstrap samples.
It means that we want to pick 11 times 1000 samples from our dataset

In [41]:
bootstrap_size, bootstrap_n = bootstrap_samples, 11
bootstrap_samples = np.random.randint(0, train_n, size = (bootstrap_n, bootstrap_size))

Plot the number of bootstrap repetition each samples appear in

In [64]:
#index, counts = np.unique(bootstrap_samples, return_counts = True)
#size_dots = np.zeros(train_n)
#size_dots[index] = counts

size_dots_rows = np.zeros(train_n)
for row in range(bootstrap_samples.shape[0]):
  index = np.unique(bootstrap_samples[row,:])
  size_dots_rows[index] += 1
    
px.histogram(x=size_dots_rows, height = 200).show()

In [75]:
# Add test

def multiple_trees_error(s_train, t_train, s_test, t_test, bootstrap_repeats, depth):
  # Create a tree for each bootstrap repeat
  clf_lst = [tree.DecisionTreeClassifier(max_depth=depth).fit(s_train[row,:], np.array(t_train)[row]) for row in bootstrap_repeats]
  
  # Precict for each sample for each tree
  tags_by_sample = [clf.predict(s_test) for clf in clf_lst]
  sum_tags = np.sum(tags_by_sample, axis = 0)
  tag_of_comitee = np.argmax([np.ones(sum_tags.shape)*bootstrap_repeats.shape[0] - sum_tags, sum_tags], axis = 0)
  return np.sum(tag_of_comitee != np.array(t_test))/(t_test.shape[0])

bootstrap_ns = list(range(2, 50, 5))
depths = list(range(5, 15, 3))

errors_lst = []
for depth in depths:
  bootsrap_errors = []
  for bootstrap_n in bootstrap_ns:
    bootstrap_repeats = np.random.randint(0, train_n, size = (bootstrap_n, bootstrap_size))
    bootsrap_errors.append(multiple_trees_error(samples_train, tags_train, samples_test, tags_test, bootstrap_repeats, depth))
  errors_lst.append(bootsrap_errors)


NameError: name 'errors_lst_arr' is not defined

In [76]:
errors_lst_arr = np.array(errors_lst)

fig = go.Figure()
for i, depth in enumerate(depths):
  errors_single = errors_lst_arr[i,:]
  fig.add_trace(go.Scatter(x=bootstrap_ns, y=errors_single,
                      mode='lines',
                      name=depth))
fig.show()